In [2]:
'''
IC272 DS3
lab 4

Shubham Shukla
B20168
'''
# importing the important libraries
import pandas as pd
import sklearn as skit
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# reading the csv file
df = pd.read_csv('SteelPlateFaults-2class.csv')

# storing all data attribute except class
x = df.iloc[:, :df.shape[1] - 1]

# storing the class attribute data
y = df.iloc[:, df.shape[1] - 1]

"""Question 1"""

# Splitting the data in train test data
[X_train, X_test, X_label_train, X_label_test] = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

# storing the train test data to external csv file
X_train.to_csv('SteelPlateFaults-train.csv')
X_test.to_csv('SteelPlateFaults-test.csv')

print("Before normalizing the result are : \n")

# making a loop that will compute confusion matrix, accuracy score for k = 1, 3, 5 respectively
for i in range(3):
    # using the inbuilt classifier
    classifier = KNeighborsClassifier(n_neighbors = 2*i + 1, metric = 'minkowski', p = 2)
    classifier.fit(X_train, X_label_train)
    
    # storing the predicted data by classifier
    y_pred = classifier.predict(X_test) 
    
    # printing the output
    print("\nFor K = ",2*i + 1 ," : ")
    print("\nconfusion matrix is -> \n",confusion_matrix(y_pred, X_label_test))
    print("\nAccuracy score is -> \n", accuracy_score(y_pred, X_label_test));
    
"""Question 2"""

# making a funtion to normalize the data
def normalize(data):
    for att in data.columns:
        data[att] = (data[att] - data[att].min())/(data[att].max() - data[att].min())
    return data;

# normalizing data and again splitting the data in train and test data
x = normalize(x);
[X_train, X_test, X_label_train, X_label_test] = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

print("\n\nAfter normalizing the result are : \n")

# Again running a loop for k = 1, 3, 5 
for i in range(3):
    # using the inbuilt classifier
    classifier = KNeighborsClassifier(n_neighbors = 2*i + 1, metric = 'minkowski', p = 2)
    classifier.fit(X_train, X_label_train)
    
    # storing the predicted data by classifier
    y_pred = classifier.predict(X_test) 
    
    # printing the output
    print("\nFor K = ",2*i + 1 ," : ")
    print("\nconfusion matrix is -> \n",confusion_matrix(y_pred, X_label_test))
    print("\nAccuracy score is -> \n", accuracy_score(y_pred, X_label_test));
    
'''Question 3'''

# making a dataframe of the training data for my convenience
X_train = (pd.DataFrame(X_train, columns = x.columns))
X_train['Class'] = X_label_train
df = X_train
df['Class'] = X_label_train

# splitting and storing the data wrt the classes
data_class0 = df[df['Class'] == 0]; 
data_class1 = df[df['Class'] == 1]

# making a function that will find the covariance matrix 
def create_cov(inp):
    ind = inp.columns
    lst = []
    cov_list = []
    for i in ind:
        for j in ind:
            cov_list.append(inp[i].cov(inp[j]))
        lst.append(cov_list)
        cov_list = []
    return lst

# storing mean vector and covariance vector wrt class
data_class0 = data_class0.drop(['Class'], axis = 1); mean_class0 = data_class0.mean()
data_class1 = data_class1.drop(['Class'], axis = 1); mean_class1 = data_class1.mean()
cov_class0 = pd.DataFrame(create_cov(data_class0), index = data_class0.columns, columns = data_class0.columns)
cov_class1 = pd.DataFrame(create_cov(data_class0), index = data_class0.columns, columns = data_class0.columns)

prior = data_class0.shape[0]/(data_class0.shape[0] + data_class1.shape[0])
# making a likelihood function viz going to return the expected unimodal values
def likelihood(data_vector, mean_vector, cov_mat):
    matrix = np.dot((data_vector-mean_vector).T, np.linalg.inv(cov_mat))
    ins = -0.5*np.dot(matrix, data_vector - mean_vector)
    ex = np.exp(ins)
    return (ex/(abs(np.linalg.det(cov_mat)))**0.5)

# making a function that will predict the class according to query vector data
def class_predicted(data_vector):
    p_0 = likelihood(data_vector, mean_class0, cov_class0)
    p_1 = likelihood(data_vector, mean_class1, cov_class1)
    if(prior*p_0 > (1- prior)*p_1): return 0;
    else : return 1;
    
# storing the predicted class in a list variable
predicted_data = []
for i in range(X_test.shape[0]):
    predicted_data.append(class_predicted(X_test.iloc[i, :]))

# printing the final output
print("The statistics of predicted class by bayes classifier is : ")
print("\nconfusion matrix is -> \n",confusion_matrix(predicted_data, X_label_test))
print("\nAccuracy score is -> \n", accuracy_score(predicted_data, X_label_test))

"""Question 4"""
print("The mean data are : ")
print("\n for class 0 : \n", mean_class0)
print("\n for class 1 : \n", mean_class1)

Before normalizing the result are : 


For K =  1  : 

confusion matrix is -> 
 [[ 81  27]
 [ 27 201]]

Accuracy score is -> 
 0.8392857142857143

For K =  3  : 

confusion matrix is -> 
 [[ 83  12]
 [ 25 216]]

Accuracy score is -> 
 0.8898809523809523

For K =  5  : 

confusion matrix is -> 
 [[ 82   9]
 [ 26 219]]

Accuracy score is -> 
 0.8958333333333334


After normalizing the result are : 


For K =  1  : 

confusion matrix is -> 
 [[104   9]
 [  4 219]]

Accuracy score is -> 
 0.9613095238095238

For K =  3  : 

confusion matrix is -> 
 [[104   6]
 [  4 222]]

Accuracy score is -> 
 0.9702380952380952

For K =  5  : 

confusion matrix is -> 
 [[103   7]
 [  5 221]]

Accuracy score is -> 
 0.9642857142857143


<ipython-input-2-f1825fa16d0e>:114: RuntimeWarning: overflow encountered in exp
  ex = np.exp(ins)


The statistics of predicted class by bayes classifier is : 

confusion matrix is -> 
 [[ 69  18]
 [ 39 210]]

Accuracy score is -> 
 0.8303571428571429
The mean data are : 

 for class 0 : 
 X_Minimum                0.080395
X_Maximum                0.165203
Y_Minimum                0.137366
Y_Maximum                0.137372
Pixels_Areas             0.047598
X_Perimeter              0.033848
Y_Perimeter              0.011358
Sum_of_Luminosity        0.069740
Minimum_of_Luminosity    0.263068
Maximum_of_Luminosity    0.457679
Length_of_Conveyer       0.318680
TypeOfSteel_A300         0.003534
TypeOfSteel_A400         0.996466
Steel_Plate_Thickness    0.000951
Edges_Index              0.127057
Empty_Index              0.476330
Square_Index             0.589849
Outside_X_Index          0.121995
Edges_X_Index            0.559508
Edges_Y_Index            0.500517
Outside_Global_Index     0.268551
LogOfAreas               0.675562
Log_X_Index              0.629985
Log_Y_Index              0.